In [1]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pickle
import scrapy
import json

with open('autotrader_make_models3.pickle', 'rb') as handle:
    make_models = pickle.load(handle)

chromrdriver = r"C:\Users\alfab\Documents\chromedriver.exe"
os.environ["webdriver.chrome.driver"] = chromrdriver

In [32]:
import pickle
with open('autotrader_make_models3.pickle', 'rb') as handle:
    make_models = pickle.load(handle)

In [3]:
#create file structure
src_folder = 'scraped_page_src'
#page_src --> make --> model ---> make_model_0.html, make_model_1.html, make_model_2.html,,,,,
for make in list(make_models.keys()):
     os.makedirs(os.path.join(src_folder,make))

for make in list(make_models.keys()):
    make_path = os.path.join(src_folder,make)
    for model in make_models[make]:
         os.makedirs(os.path.join(make_path,model.replace('/','_')))

In [37]:
target_url = get_url(make_models['toyota'][2],0)
chromrdriver = r"C:\Users\alfab\Documents\chromedriver.exe"
os.environ["webdriver.chrome.driver"] = chromrdriver
driver = webdriver.Chrome(chromrdriver)
driver.get(target_url)

In [36]:
makemodel = make_models['Toyota'][4]
get_url(makemodel,location,2018,2021)

'https://www.autotrader.com/cars-for-sale/all-cars/toyota/camry/washington-dc-20500?dma=&maxMileage=200000&searchRadius=0&location=&marketExtension=include&endYear=2021&startYear=2018&isNewSearch=true&showAccelerateBanner=false&sortBy=relevance&numRecords=100'

In [92]:
# def get_url(makemodel,page_num):
#     base='https://www.autotrader.com/cars-for-sale/all-cars/'
#     PageNumberString = str(page_num*100)
#     remainder = 'washington-dc-20500?dma=&maxMileage=200000&searchRadius=0&location=&marketExtension=include&isNewSearch=false&showAccelerateBanner=false&sortBy=relevance&numRecords=100&firstRecord='+PageNumberString
#     return(base+makemodel+remainder)
location='washington-dc-20500'
def get_url(makemodel,location,startyear,endyear):
    base_url = 'https://www.autotrader.com/cars-for-sale/all-cars/'
    location_text = location+'?dma=&maxMileage=200000&searchRadius=0&location=&marketExtension=include&'
    date_range = 'endYear='+str(endyear)+'&startYear='+str(startyear)
    page_info ='&isNewSearch=true&showAccelerateBanner=false&sortBy=relevance&numRecords=100'
    return(base_url+makemodel+location_text+date_range+page_info)

def single_scrape(makemodel,make,location,startyear,endyear):
    target_folder = os.path.join('scraped_page_src',make)
    model_folder = os.path.join(target_folder,makemodel.replace('/','_'))
    
    target_url = get_url(makemodel,location,startyear,endyear)
    chromrdriver = r"C:\Users\alfab\Documents\chromedriver.exe"
    os.environ["webdriver.chrome.driver"] = chromrdriver
    driver = webdriver.Chrome(chromrdriver)
    driver.get(target_url)
    
    file = open(os.path.join(model_folder,makemodel.replace('/','_')+str(startyear)+'-'+str(endyear)+'.html'),'w',encoding = 'utf-8')
    file.write(driver.page_source)
    file.close()
#     time.sleep(6)
    driver.close()
    

In [ ]:
https://www.autotrader.com/cars-for-sale/all-cars/acura/
    
0498c-mailers-s-99-99999?dma=&searchRadius=0&isNewSearch=true&marketExtension=include&showAccelerateBanner=false&sortBy=relevance&numRecords=25

In [91]:
single_scrape(make_models['Toyota'][4],'Toyota', location, 2015,2020)
print(get_url(make_models['Toyota'][4], location, 2015,2020))

https://www.autotrader.com/cars-for-sale/all-cars/toyota/camry/0498c-mailers-s-99-99999?dma=&maxMileage=200000&searchRadius=0&location=&marketExtension=include&endYear=2020&startYear=2015&isNewSearch=true&showAccelerateBanner=false&sortBy=relevance&numRecords=100


In [93]:
all_makes = list(make_models.keys())
startyear1 = 2005
endyear1 = 2014
location = 'washington-dc-20500'
startyear2 = 2015
endyear2 = 2021

for make in all_makes[0:1]:
    for model in make_models[make]:
        single_scrape(model,make,location,startyear1,endyear1)
        single_scrape(model,make,location,startyear2,endyear2)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=85.0.4183.121)


In [89]:
all_makes[0:1]

['Acura']

In [76]:
all_makes[0:7]

['Acura', 'Audi', 'BMW', 'Buick', 'Cadillac', 'Chevrolet', 'Chrysler']

In [77]:
all_makes[7:14]

['Dodge', 'Ferrari', 'Ford', 'Genesis', 'GMC', 'Honda', 'Hyundai']

In [78]:
all_makes[14:21]

['INFINITI', 'Jaguar', 'Jeep', 'Kia', 'Land_Rover', 'Lexus', 'Lincoln']

In [81]:
all_makes[21:28]

['MAZDA', 'Mercedes-Benz', 'MINI', 'Mitsubishi', 'Nissan', 'Porsche', 'RAM']

In [84]:
all_makes[28:]

['Subaru', 'Tesla', 'Toyota', 'Volkswagen', 'Volvo']

In [91]:
def extract_data(target_page):
    data = open(target_page,'r',encoding='utf-8')
    soup = BeautifulSoup(data, 'html.parser')
    all_listings = soup.find_all("div",{"data-cmp":"inventoryListing"})

    names = []
    prices = []
    milage_infos = []

    for i in range(len(all_listings)):
        try:
            price = all_listings[i].find_all("span", {"class":"first-price"})[0].text
        except:
            price ='na'
        try:
            milage = all_listings[i].find_all("div", {"class":"item-card-specifications col-xs-9 margin-top-4 text-subdued-lighter margin-bottom-3"})[0].text
        except:
            milage = 'na'
        try:
            name = all_listings[i].find_all("div", {"class":"display-flex justify-content-between"})[0].text
        except:
            name='na'


        names.append(name)
        prices.append(price)
        milage_infos.append(milage)
    return({"names":names,"prices":prices,"milages":milage_infos})

In [101]:
make = 'Acura'
startyear1 = 2005
endyear1 = 2014
location = 'washington-dc-20500'
startyear2 = 2015
endyear2 = 2021

makemodel = make_models[make][1]
target_folder = os.path.join('scraped_page_src',make)
model_folder = os.path.join(target_folder,makemodel.replace('/','_'))
target_page = os.path.join(model_folder,makemodel.replace('/','_')+str(startyear1)+'-'+str(endyear1)+'.html')

In [103]:
data = open(target_page,'r',encoding='utf-8')
soup = BeautifulSoup(data, 'html.parser')

In [151]:
all_listings = soup.find_all("div",{"data-cmp":"inventoryListing"})

names = []
prices = []
milage_infos = []

for i in range(len(all_listings)):
    try:
        price = all_listings[i].find_all("span", {"class":"first-price"})[0].text
    except:
        price ='na'
    try:
        milage = all_listings[i].find_all("div", {"class":"item-card-specifications col-xs-9 margin-top-4 text-subdued-lighter margin-bottom-3"})[0].text
    except:
        milage = 'na'
    try:
        name = all_listings[i].find_all("div", {"class":"display-flex justify-content-between"})[0].text
    except:
        name='na'
    

    names.append(name)
    prices.append(price)
    milage_infos.append(milage)

In [139]:
i= 0
price = all_listings[i].find_all("span", {"class":"first-price"})[0].text

In [130]:
soup.find_all("div",{"data-cmp":"inventoryListing"})[0].find_all("span", {"class":"first-price"})[0].text

'15,590'

In [131]:
soup.find_all("div",{"data-cmp":"inventoryListing"})[0].find_all("div", {"class":"item-card-specifications col-xs-9 margin-top-4 text-subdued-lighter margin-bottom-3"})[0].text


'62,569 milesColor: SilverMPG: 24 City / 34 HighwayDrive Type: 2 wheel drive - frontEngine: 4-Cylinder'

In [132]:
soup.find_all("div",{"data-cmp":"inventoryListing"})[0].find_all("div", {"class":"display-flex justify-content-between"})[0].text

'Used 2013 Acura ILX w/ Premium Package'